<a href="https://colab.research.google.com/github/thebayn/desertfight/blob/main/kohya_ss_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kohya SS WebUI Colab Setup

This Colab workbook sets up a Kohya SS instance on Colab and provides a link to access the Kohya WebUI on Gradio Live. Kohya SS is a Python library that provides Stable Diffusion-based models for image, text, and audio generation tasks. This Colab workbook provides a convenient way for users to run Kohya SS without needing to install anything on their local machine.

This workbook was inspired by the work of [Spaceginner](https://github.com/Spaceginner)'s original Colab workbook and the [Kohya SS project](https://github.com/bmaltais/kohya_ss) by [bmaltais](https://github.com/bmaltais). The Colab workbook was coded by [panguin6010](https://github.com/panguin6010)


## Tutorials

Before running this code, make sure you are familiar with using Colab workbooks and have a basic understanding of Kohya SS and its usage. You can find tutorials for these online. If you encounter any issues or have suggestions for improvement, feel free to contribute to the project.





---



In [8]:
#@markdown #Step 1: Mounting Google Drive

#@markdown The first step in setting up Kohya SS on Colab is to mount your Google Drive to the Colab notebook. This allows you to save and access files from your Google Drive in the Colab notebook.

#@markdown To mount your Google Drive, run the following code block, which mounts your Google Drive to the /content/gdrive directory in the Colab notebook.



from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).




---



In [9]:
#@markdown #Kohya SS WebUI Installation

#@markdown Now that your Google Drive is linked, we need to install the Kohya SS WebUI.

#@markdown The code clones the [Kohya SS Google Colab]("https://github.com/panguin6010/kohya_ss_google_colab") repository and creates the necessary directories for Kohya SS to run. It then resets the git repository and pulls the latest changes. Finally, it displays a success message.

#@markdown Note: If Google Drive is not connected, the code will use Colab storage instead.

#@title
# Import necessary libraries
from IPython.display import clear_output
from IPython.utils import capture
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

# WebUI Installation

# Check if Google Drive is connected
if not os.path.exists("/content/gdrive/MyDrive/"):
    print("Gdrive not connected, using colab storage ...")
    time.sleep(4)
    !mkdir -p /content/gdrive/MyDrive/

# Clone the repository and create necessary directories
with capture.capture_output() as cap:
    def inf(msg, style, wdth):
        inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
        display(inf)

    %mkdir -p /content/gdrive/MyDrive/sd
    %cd /content/gdrive/MyDrive/sd
    !git clone https://github.com/panguin6010/kohya_ss_google_colab kohya_ss_colab
    !mkdir -p /content/gdrive/MyDrive/sd/kohya_ss_colab/cache/huggingface
    !ln -s /content/gdrive/MyDrive/sd/kohya_ss_colab/cache/huggingface /root/.cache/

# Reset the git repository and pull the latest changes
with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/sd/kohya_ss_colab/
    !git reset --hard
    time.sleep(1)

print("Updating the repository...")
!git pull

# Clear the output and display the success message
clear_output()
inf("✓ Done", "success", "50px")

Button(button_style='success', description='✓ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

---

In [10]:
#@markdown #Requirements Installation

#@markdown Now that we have downloaded the Kohya SS WebUI, we need to install the necessary requirements.

# Print the status message
print("Installing requirements...")

# Change the working directory to the project folder
%cd /content/gdrive/MyDrive/sd/kohya_ss_colab/

# Install the requirements
with capture.capture_output() as cap:
    # Uncomment the following line if you need to install specific versions of torch and torchvision
    # !pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

    # Install the requirements from the requirements.txt file
    !pip install -r requirements.txt

# Clear the output to keep the notebook clean
clear_output()

# Print the success message
inf("✓ Done", "success", "50px")

Button(button_style='success', description='✓ Done', disabled=True, layout=Layout(min_width='50px'), style=But…


---



In [11]:
#@markdown # Start Kohya ss WebUI

User = "compents" #@param {type:"string"}
Password = "SethJo@1999" #@param {type:"string"}

#@markdown - Adding a username and password is not necessary but it will improve the security of your Kohya instance.
#@markdown ______
#@markdown # Please click the link that concludes with ```gradio.live``` to access your instance
# Encourage users to contribute improvements
print("Please feel free to make any changes or improvements you think would enhance this setup. Your input and contributions are greatly appreciated!")
# Check if the user has provided a username and password
if User and Password:
    # Run the Kohya GUI with the provided credentials
    !python /content/gdrive/MydDrive/sd/kohya_ss_colab/kohya_gui.py --username $User --password $Password --share
else:
    # Run the Kohya GUI without credentials
    !python /content/gdrive/MyDrive/sd/kohya_ss_colab/kohya_gui.py --share


Please feel free to make any changes or improvements you think would enhance this setup. Your input and contributions are greatly appreciated!
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/sd/kohya_ss_colab/kohya_gui.py", line 4, in <module>
    from dreambooth_gui import dreambooth_tab
  File "/content/gdrive/MyDrive/sd/kohya_ss_colab/dreambooth_gui.py", line 13, in <module>
    from library.common_gui import (
  File "/content/gdrive/MyDrive/sd/kohya_ss_colab/library/common_gui.py", line 2, in <module>
    from easygui import msgbox
ModuleNotFoundError: No module named 'easygui'
